<a href="https://colab.research.google.com/github/nikhilnimbalkar1/STS/blob/master/STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STS prediction using trained models 

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_hub as hub 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

# **Load trained models**
We have two models trained on different datasets to choose from.


In [0]:
model=tf.keras.models.load_model("/content/gdrive/My Drive/Colab Notebooks/STS_models/saved_model_sick.h5")

**Load pretrained USE model**

In [0]:
def load_USE():
  use_model_url="https://tfhub.dev/google/universal-sentence-encoder-large/5"
  embed=hub.load(use_model_url)
  return embed

In [0]:
embed=load_USE()
def create_embeddings(sentence):
    embedding = tf.nn.l2_normalize(embed(sentence), axis=1)
    embedding=tf.expand_dims(embedding,axis=2)
    return embedding

# Question-Question Pair dataset
We'll predict semantic similarity on the question pair dataset by Quora.
The Quora dataset does not include actual similarity score but rather just a 0 and 1 score to check for duplicates.We'll predict our scores ranging from 0 to 1. 

In [0]:
questions=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Datasets_STS/questions.csv",usecols=['question1','question2','is_duplicate'])

**Prepare embeddings dataset for our model**

In [0]:
def prepare_data(sts_data):
  sts_encode1 = create_embeddings(tf.constant(sts_data.iloc[:,0].tolist()))
  sts_encode2 = create_embeddings(tf.constant(sts_data.iloc[:,1].tolist()))
  sts_sims=sts_data.iloc[:,2]
  sts_sims=tf.convert_to_tensor(sts_sims)
  return sts_encode1,sts_encode2,sts_sims

**Prediction and saving file to drive**

In [0]:
def predict_samples(samples,model):
  samples_embeddings1,samples_embeddings2,samples_similarity=prepare_data(samples)
  preds=model.predict([samples_embeddings1,samples_embeddings2])
  preds=preds.tolist()
  preds=np.squeeze(preds,axis=1)
  preds=pd.DataFrame(preds)
  samples['predicted_sim']=preds
  return samples

In [0]:
predicted_questions=predict_samples(questions[:100],model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
predicted_questions.to_csv("/content/gdrive/My Drive/Colab Notebooks/Datasets_STS/qqq_sts.csv")

# Test your own input
Provide your own input to predict similarity in percentage.

In [0]:
inp1=input("First Sentence/Paragraph: \n\t")
inp2=input("Second Sentence/Paragraph: \n\t")

In [0]:
sent1=create_embeddings([inp1])
sent2=create_embeddings([inp2])

In [0]:
sem_sim=float(model.predict([sent1,sent2]))
print("Semantic Similarity:  {:05.2f}".format(sem_sim*100),"%")